### Load modules and specify file path

In [187]:
# Import required modules
import pandas as pd
import glob2
import re
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Specify paths to files
raw_data_path = r'C:\Users\MT1070\Desktop\Master Call Volume\raw-report-extracts' 

# Display names of files in master folder
raw_files = glob2.glob(raw_data_path + "/*.xlsx")

# Initialize an empty data frame to store data from all files
final_sheet = pd.DataFrame()

### Read Data Set and Configure Entry Date 

In [ ]:
# Iteratively read data extracts from master files and append them to the final DataFrame
data_frames = []  # Store DataFrames from master files
for file in raw_files:
    try:
        df = pd.read_excel(file, sheet_name=None, skipfooter=2, engine='openpyxl')
        df_concat = pd.concat(df.values(), ignore_index=True, sort=False)
        data_frames.append(df_concat)
    except Exception as e:
        print(f"Error occurred while processing file: {file}")
        print(str(e))

## Reset Index and Set Column Headers

In [ ]:
if len(data_frames) > 0:
    final_sheet = pd.concat(data_frames, ignore_index=True)

# Set desired column as 

final_sheet.reset_index(drop=True, inplace=True)

# Define headers
final_sheet.columns = final_sheet.iloc[0]
final_sheet = final_sheet[1:]


In [ ]:
final_sheet['department_name']

### Extract Date from Datetime column

In [ ]:
# Print the values in the "datetime" column
column_name = 'datetime'
if column_name.strip() in final_sheet.columns:
    final_sheet[column_name.strip()] = pd.to_datetime(final_sheet[column_name.strip()], format='%m/%d/%y %I:%M:%S %p', errors='coerce')

    # Check the data type of the "datetime" column after conversion
    datetime_column = final_sheet[column_name.strip()]

    # Create the 'date' column by extracting month/date/year values
    final_sheet['date'] = final_sheet[column_name.strip()].dt.strftime('%m/%d/%Y')

    # Print the datetime entries that cannot be parsed
    invalid_entries = final_sheet[datetime_column.isna()][column_name.strip()]
    print(f"Invalid datetime entries:")
    print(invalid_entries)

else:
    print(f"Column '{column_name.strip()}' not found in the DataFrame.")


### Count Entries for Successful & Unsuccessful Calls Transferred

In [ ]:
# Helpful Tools
def eval_data_type(column):
    data_types = column.apply(type).unique()
    
    # Print the list of data types found
    print("Data types found in the column:")
    for data_type in data_types:
        print(data_type)

    # usage: eval_data_type(final_sheet['valid_department_transfers'])


# Count Holistic Metrics
total_internal_calls = final_sheet['internal_or_external'].str.count('10030').sum()
total_external_calls = final_sheet['internal_or_external'].str.count('10040').sum()
total_calls = total_internal_calls + total_external_calls

output_holistic_metrics = False
if output_holistic_metrics:
    print(f"Total number of internal calls: {total_internal_calls}")
    print(f"Total number of external calls: {total_external_calls}")
    print(f"Total number of calls: {total_calls}\n\n")


# Count number of successful workflows
first_try_success = final_sheet['department_utternace_first_try_success'].count()
second_try_success = final_sheet['department_utternace_second_try_success'].count()
third_try_success = final_sheet['department_utternace_second_try_success'].count()
total_success = first_try_success + second_try_success + third_try_success


# Output Successful Counts
output_successful_counts = True
if output_successful_counts:
    print(f"Number of entries in First Try Success: {first_try_success}")
    print(f"Number of entries in Second Try Success: {second_try_success}")
    print(f"Number of entries in Third Try Success: {third_try_success}")
    print(f"Total number of successful call transfers: {total_success}\n")


# Count number of unsuccessful workflows
first_nomatch = final_sheet['department_utternace_first_nomatch'].count()
second_nomatch = final_sheet['department_utternace_second_nomatch'].count()
third_nomatch = final_sheet['department_utternace_third_nomatch'].count()
final_nomatch = final_sheet['department_utternace_max_nomatch'].count()
total_nomatch = first_nomatch + second_nomatch + third_nomatch + final_nomatch

# Output Usuccessful Counts
output_unsuccessful_counts = False
if output_unsuccessful_counts:
    print(f"Number of unsuccessful first try entries: {first_nomatch}")
    print(f"Number of unsuccessful second try entries: {second_nomatch}")
    print(f"Number of unsuccessful third try entries: {third_nomatch}")
    print(f"Total number of unsuccessful calls: {total_nomatch}")



### SpinSci Metrics Verification

In [192]:
# SpinSci Metric Verification
# What to expect from the dataset
eval_data_type(final_sheet['call_transfer_success'])

# Count Successful and Unsuccessful Calls
spinsci_success_count = int(final_sheet['call_transfer_success'].str.count('10050').sum())
non_transfers = final_sheet['call_transfer_success'].isna().sum()
verify_with_total_calls = spinsci_success_count + non_transfers

# Output results
print(f"Total number of calls based on internal and external entries: {total_calls}")
print(f"Calculated total number of calls: {verify_with_total_calls}")
print(f"Number of successful transfers: {spinsci_success_count}")
print(f"Number of unsuccessful transfers: {non_transfers}\n\n")

# Extract dataframe with metrics of all successful calls
success_filter = final_sheet['call_transfer_success'].str.contains('10050') & final_sheet['call_transfer_success'].notna()
successful_calls = final_sheet[success_filter].copy()


# Count the number of Operator requests
option2_name = '(menuoption2)'
option3_name = '(menuoption3)'
option4_name = '(menuoption4)'
option5_name = '(menuoption5)'
option0_name = '(press 0)'
trailing_operator_pattern = r'^operator(?!.*\(.*\))'

list_of_options = [option2_name, option3_name, option4_name, option5_name, option0_name]

option2_selection = successful_calls['department_name'].str.count(option2_name).sum()
option3_selection = successful_calls['department_name'].str.count(option3_name).sum()
option4_selection = successful_calls['department_name'].str.count(option4_name).sum()
option5_selection = successful_calls['department_name'].str.count(option5_name).sum()
option0_selection = successful_calls['department_name'].str.count(option0_name).sum()

print(f"Total number of callers selecting option 2: {option2_selection}")
print(f"Total number of callers selecting option 3: {option3_selection}")
print(f"Total number of callers selecting option 4: {option4_selection}")
print(f"Total number of callers selecting option 5: {option5_selection}")
print(f"Total number of callers selecting option 0: {option0_selection}")

# Count the number of non-Operator requests
non_operator_requests = (~successful_calls['department_name'].isin(list_of_options)).sum()
requests_total = non_operator_requests + option2_selection + option3_selection + option4_selection + option5_selection + option0_selection
print(f"The number of non-Operator requests were: {non_operator_requests}")
print(f"Total number of calls based on call breakdown: {requests_total}")

print(successful_calls['department_name'])

spinsci_calc_metrics = False
if spinsci_calc_metrics:
    spinsci_percent_success_transfers = int((spinsci_success_count/total_calls)*100)
    calc_percent_success = int((total_success/total_calls)*100)

    print(f"Total number of SpinSci ID'd successful call transfers: {spinsci_success_count}")
    print(f"Percentage of successful calls transfered: {spinsci_percent_success_transfers}%")
    print(f"Calc. Percentage of successful calls transfered: {calc_percent_success}%\n\n")


KeyError: 'call_transfer_success'

### Review Datatypes in department_name column

In [ ]:
float_values = final_sheet.loc[final_sheet['department_name'].apply(lambda x: isinstance(x, float)), 'department_name']

print("Float values in the column:")
print(float_values)

### Change department_name entries format

In [ ]:
# String manipulation to modify department_name entries
count_department_tranfers = final_sheet['department_name'].count()
print(f"The total number entries in Department Name column: {count_department_tranfers}\n")
print("Breakdown of Department Transfers Column:")

# Count the number of NaN entries in department_name
print(f"Total number of calls: {total_calls}\n\n")
nonconclusive_transfers = final_sheet['department_name'].isna().sum()
print(f"Number of non-transfers: {nonconclusive_transfers}")

# Create a new column with non-NaN entries from 'column_name'
final_sheet['valid_department_transfers'] = final_sheet['department_name'].fillna('')

# Change department_name to str type
final_sheet['valid_department_transfers'] = final_sheet['valid_department_transfers'].astype(str)
# dtype = final_sheet['valid_department_transfers'].dtype
# print(f"The format for department_name column: {dtype}")

# Count number of entries without "Operator"
count_non_operator = final_sheet['valid_department_transfers'].str.contains('Operator', case=False, na=False).sum()
# count_no

print(f"Number of requests that are not the Operator {count_non_operator}")

# Print the DataFrame to see the updated column
# print(final_sheet['valid_department_transfers'])


operator_selections = final_sheet['valid_department_transfers'].str.count('Operator').sum()
print(f"Total number of Operator Selections: {operator_selections}\n\n")
# misc_selection = final_sheet['department_name'].str.count('Operator(press 0)').sum()
# count_department_tranfers = final_sheet['department_name'].count()


# Department Column Transfer Check. Total same as spinsci_success_count
# final_sheet['department_name'] = final_sheet['department_name'].astype('str')

    # Verify the data type of entries in the target column
# print(final_sheet['department_name'].dtype)

# unique_values = final_sheet['department_name'].unique()
# print(unique_values)

### Graveyard

### Locate Invalid Entries 

In [ ]:
column_name = 'datetime'
if column_name.strip() in final_sheet.columns:
    final_sheet[column_name.strip()] = pd.to_datetime(final_sheet[column_name.strip()], format='%m/%d/%y %I:%M:%S %p', errors='coerce')

    # Check the data type of the "datetime" column after conversion
    datetime_column = final_sheet[column_name.strip()]
    data_type = datetime_column.dtype
    print(f"The data type of elements in the '{column_name}' column after conversion is: {data_type}")

    # Print the datetime entries that cannot be parsed
    invalid_entries = final_sheet[datetime_column.isna()][column_name.strip()]
    print(f"Invalid datetime entries:")
    print(invalid_entries)

# Utilize the output of the script above to find the NaN values.
index_value = 2978
print(final_sheet.iloc[index_value])

In [ ]:
def convert_to_datetime(df, column_name):
    if column_name in df.columns:
        if column_name == 'datetime':
            try:
                df['Date'] = df[column_name].dt.date
                # df[column_name] = pd.to_datetime(df[column_name], format='%m/%d/%y %I:%M:%S %p')
                return df
            except ValueError:
                print(f"Invalid datetime format in column {column_name}:")
                print(df[column_name])
        else:
            df[column_name] = pd.to_datetime(df[column_name], format='%m/%d/%y %I:%M:%S %p')
            return df
    else:
        print(f"Column '{column_name} not found in Dataframe.")
        return None
    
final_sheet = convert_to_datetime(final_sheet, 'datetime')

# print(final_sheet)